In [2]:
from IPython.core.magic import register_cell_magic

@register_cell_magic
def write_and_run(line, cell):
    """Write command to file before executing on notebook"""
    args = line.split()
    append = ''
    if '-a' in args:
        append = args.pop()
    for arg in args:
        file = arg
        mode = 'w' if append != '-a' else 'a'
        with open(file, mode) as f:
            f.write(f"{cell}\n")
           
    get_ipython().run_cell(cell)

In [4]:
!pip install -r ../requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 15.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 17.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 11.3 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 14.5 MB/s eta 0:00:0000:0100:01
  Created wheel for SQLAlchemy: filename=SQLAlchemy-1.3.24-cp310-cp310-linux_x86_64.whl size=1203249 sha256=f05fdfc6f23017e3560e55515279889fac20209986b88360841c4f6c2272684f
  Stored in directory: /home/jovyan/.cache/pip/wheels/27/51/b3/3481e88d5a5ba95dd4aafedc9316774d941c4ba61cfb93add8
Successfully built SQLAlchemy
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 1.4.40
    Uninstalling SQLAlchemy-1.4.40:
      Successfully uninstalled SQLAlchemy-1.4.40


In [5]:
import os

import pandas as pd

from dotenv import dotenv_values
from sqlalchemy import create_engine, inspect

In [6]:
CONFIG = dotenv_values('.env')
if not CONFIG:
    CONFIG = os.environ

connection_uri = "postgresql+psycopg2://{}:{}@{}:{}".format(
    CONFIG["POSTGRES_USER"],
    CONFIG["POSTGRES_PASSWORD"],
    CONFIG['POSTGRES_HOST'],
    CONFIG["POSTGRES_PORT"],
)

In [7]:
engine = create_engine(connection_uri, pool_pre_ping=True)
engine.connect()

In [8]:
# Extract
dataset = "https://gist.githubusercontent.com/JoeLeavitt/f9d1e14e87f2ca41609b0af63fbab7af/raw/9fedfd46068bdf6ee62731da4cf08c56df7c4866/DATA.csv"

In [9]:
df = pd.read_csv(dataset)

In [10]:
df.head()

,id,first_name,last_name,email,gender,ip_address
0,1,Margaretta,Laughtisse,mlaughtisse0@mediafire.com,Genderfluid,34.148.232.131
1,2,Vally,Garment,vgarment1@wisc.edu,Bigender,15.158.123.36
2,3,Tessa,Curee,tcuree2@php.net,Bigender,132.209.143.225
3,4,Arman,Heineking,aheineking3@tuttocitta.it,Male,157.110.61.233
4,5,Roselia,Trustie,rtrustie4@ft.com,Non-binary,49.55.218.81


In [11]:
df.describe()

,id
count,1000.000000
mean,500.500000
std,288.819436
min,1.000000
25%,250.750000
50%,500.500000
75%,750.250000
max,1000.000000


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          1000 non-null   int64 
 1   first_name  1000 non-null   object
 2   last_name   1000 non-null   object
 3   email       1000 non-null   object
 4   gender      1000 non-null   object
 5   ip_address  1000 non-null   object
dtypes: int64(1), object(5)
memory usage: 47.0+ KB


In [13]:
raw_table_name = 'raw_data'
df.to_sql(raw_table_name, engine, if_exists='replace')

In [14]:
def check_table_exists(table_name, engine):
    if table_name in inspect(engine).get_table_names():
        print(f"{table_name!r} exists in the DB!")
    else:
        print(f"{table_name} does not exist in the DB!")

In [15]:
check_table_exists(raw_table_name, engine)

'raw_data' exists in the DB!


In [26]:
pd.read_sql(f"SELECT * FROM {raw_table_name}", engine)

,index,id,first_name,last_name,email,gender,ip_address
0,0,1,Margaretta,Laughtisse,mlaughtisse0@mediafire.com,Genderfluid,34.148.232.131
1,1,2,Vally,Garment,vgarment1@wisc.edu,Bigender,15.158.123.36
2,2,3,Tessa,Curee,tcuree2@php.net,Bigender,132.209.143.225
3,3,4,Arman,Heineking,aheineking3@tuttocitta.it,Male,157.110.61.233
4,4,5,Roselia,Trustie,rtrustie4@ft.com,Non-binary,49.55.218.81
...,...,...,...,...,...,...,...
995,995,996,Roxana,Prandi,rprandirn@ox.ac.uk,Agender,227.67.205.244
996,996,997,Matthus,Chadwin,mchadwinro@hibu.com,Female,189.120.124.155
997,997,998,Artur,Fairbanks,afairbanksrp@live.com,Bigender,246.169.244.242
998,998,999,Amalee,Bingall,abingallrq@yandex.ru,Genderqueer,16.201.179.189


In [55]:
df = pd.read_csv(dataset)
df = df.drop_duplicates()
email_idx = df.columns.get_loc("email")
email_domains = []
print(email_idx)
for email in df['email']:
    slice_at = email.index('@') + 1
    email_domains.append(email[slice_at:])

df.insert(email_idx+1, "email_domain", email_domains, allow_duplicates=True)   
df

3


,id,first_name,last_name,email,email_domain,gender,ip_address
0,1,Margaretta,Laughtisse,mlaughtisse0@mediafire.com,mediafire.com,Genderfluid,34.148.232.131
1,2,Vally,Garment,vgarment1@wisc.edu,wisc.edu,Bigender,15.158.123.36
2,3,Tessa,Curee,tcuree2@php.net,php.net,Bigender,132.209.143.225
3,4,Arman,Heineking,aheineking3@tuttocitta.it,tuttocitta.it,Male,157.110.61.233
4,5,Roselia,Trustie,rtrustie4@ft.com,ft.com,Non-binary,49.55.218.81
...,...,...,...,...,...,...,...
995,996,Roxana,Prandi,rprandirn@ox.ac.uk,ox.ac.uk,Agender,227.67.205.244
996,997,Matthus,Chadwin,mchadwinro@hibu.com,hibu.com,Female,189.120.124.155
997,998,Artur,Fairbanks,afairbanksrp@live.com,live.com,Bigender,246.169.244.242
998,999,Amalee,Bingall,abingallrq@yandex.ru,yandex.ru,Genderqueer,16.201.179.189


In [71]:
df = pd.read_csv(dataset)
df.head()

,id,first_name,last_name,email,gender,ip_address
0,1,Margaretta,Laughtisse,mlaughtisse0@mediafire.com,Genderfluid,34.148.232.131
1,2,Vally,Garment,vgarment1@wisc.edu,Bigender,15.158.123.36
2,3,Tessa,Curee,tcuree2@php.net,Bigender,132.209.143.225
3,4,Arman,Heineking,aheineking3@tuttocitta.it,Male,157.110.61.233
4,5,Roselia,Trustie,rtrustie4@ft.com,Non-binary,49.55.218.81


In [72]:
def transform(df):
    email_idx = df.columns.get_loc("email")
    email_domains = []
    df = df.drop_duplicates()
    
    for email in df['email']:
        slice_at = email.index('@') + 1
        email_domains.append(email[slice_at:])
    
    df.insert(email_idx+1, "email_domain", email_domains, allow_duplicates=True)
    
    return df

In [73]:
df = transform(df)
df.head()

,id,first_name,last_name,email,email_domain,gender,ip_address
0,1,Margaretta,Laughtisse,mlaughtisse0@mediafire.com,mediafire.com,Genderfluid,34.148.232.131
1,2,Vally,Garment,vgarment1@wisc.edu,wisc.edu,Bigender,15.158.123.36
2,3,Tessa,Curee,tcuree2@php.net,php.net,Bigender,132.209.143.225
3,4,Arman,Heineking,aheineking3@tuttocitta.it,tuttocitta.it,Male,157.110.61.233
4,5,Roselia,Trustie,rtrustie4@ft.com,ft.com,Non-binary,49.55.218.81
